In [1]:
%matplotlib inline
import os
from sklearn.feature_extraction.text import TfidfVectorizer
from nltk.stem.snowball import SnowballStemmer
stemmer = SnowballStemmer("english")
import nltk
import json
from nltk.tokenize import sent_tokenize, word_tokenize
import re
from nltk.corpus import brown, stopwords
from nltk.cluster.util import cosine_distance
import numpy as np
import networkx as nx
import pandas as pd
from sklearn.metrics.pairwise import cosine_similarity
import pickle

In [3]:
word_embeddings = {}
f = open('../glove.6B.100d.txt', encoding='utf-8')
for line in f:
    values = line.split()
    word = values[0]
    coefs = np.asarray(values[1:], dtype='float32')
    word_embeddings[word] = coefs
f.close()

In [2]:
with open('./embeddings/embeddings.pkl', 'rb') as handle:
    word_embeddings = pickle.load(handle)

In [32]:
f=open('../sermons/Charge that to my account.txt', "r")
cttma = f.read()

In [3]:
def stem_words(doc):
    stemmed_doc = ''
    for w in doc.split(' '):
        stemmed_doc += stemmer.stem(w) + ' '
    return stemmed_doc
def remove_stopwords(sen):
    stop_words = stopwords.words('english')
    sen_new = " ".join([i for i in sen if i not in stop_words])
    return sen_new
def remove_symbols(doc):
    cleaned_doc = []
    punctuation = ['.', '..', '...', '?', ':', '!', ',', '\'', ';', '``']
    for word in doc:
        if word not in punctuation: 
            cleaned_doc.append( re.sub(r"[,.\'?!]", "", word))
    return cleaned_doc
def clean_doc(doc):
    words = re.sub(r'\'', '', doc).lower()
    words = word_tokenize(re.sub(r'\.(?=[^ \W\d])', '. ', words))
    words = remove_symbols(words)
    return words

In [6]:
data = []
for sermon in os.listdir('../sermons/preachingtoday'):
    f=open('../sermons/preachingtoday/'+ sermon, 'rb')
    contents =f.read()
    contents = json.loads(contents)
    if not contents['summary']:
        summary = clean_and_send(contents["text"])
        contents['summary'] = summary
        print(sermon)
        doc = json.dumps(contents)
        filename = '../sermons/preachingtoday/' + sermon
        with open(filename,'w') as f:
            f.write(doc)

Measurements_of_the_Heart.json
Memorial_Service_for_Bea_Campbell.json
Memorial_Service_for_Heather_Gillian.json
Memorial_Service_for_Robert_Huizenga.json
Messiahs_Community.json
Mind_Renewal.json
Modern_Golden_Calves.json
Obedience_at_the_Burning_Bush.json
Preaching_that_Reaches_the_Heart.json
Recognizing_Divine_Interruptions.json
Regaining_Lost_Joy.json
Release_Resentment.json
Renouncing_Bitterness.json
Right_on_the_Money.json
Right_Smack_in_the_Middle_of_Sin.json
Room_to_Breathe.json
Setting_the_Record_Crooked.json
Songs_of_Our_Tears.json
Speed_Dial.json
Spiritual_Growth__My_Job_or_Gods_.json
Starting_Over.json
Strong_Grace_Hard_Word_Good_Memories.json
Suffering.json
The_Audacity_to_Hope.json
The_Breath_of_God.json
The_Critical_Transaction.json
The_Forgiveness_Factor.json
The_Future_and_Forgetting.json
The_Good_News_Is_the_Bad_News_Is_Wrong.json
The_Greater_Glory.json
The_Journey_to_Integrity.json
The_King_Whose_Scepter_is_a_Towel.json
The_Most_Neglected_Commandment_in_the_Bible.json

In [4]:
def clean_and_send(doc):
    regex = re.compile('[^a-zA-Z0-9.?!\-: ]')
    doc = re.sub(r'[^a-zA-Z0-9.?!\-: ]', '' ,doc)
    doc = re.sub(r'\.(?=[^ \W\d])', '. ', doc)

    og_sentences = sent_tokenize(doc)
    sentences = sent_tokenize(doc)
    if len(sentences)< 9:
        return doc
    else:
        return summarize(sentences, og_sentences)
    
def summarize(sentences, og_sentences):
    # remove punctuations, numbers and special characters
    clean_sentences = pd.Series(sentences).str.replace("[^a-zA-Z]", " ")
    # make alphabets lowercase
    clean_sentences = [s.lower() for s in clean_sentences]
    # remove stopwords from the sentences
    clean_sentences = [remove_stopwords(r.split()) for r in clean_sentences]
    sentence_vectors = []
    for i in clean_sentences:
        if len(i) != 0:
            v = sum([word_embeddings.get(w, np.zeros((128,))) for w in i.split()])/(len(i.split())+0.001)
        else:
            v = np.zeros((128,))
        sentence_vectors.append(v)
    # similarity matrix
    sim_mat = np.zeros([len(sentences), len(sentences)])
    for i in range(len(sentences)):
        for j in range(len(sentences)):
            if i != j:
                sim_mat[i][j] = cosine_similarity(sentence_vectors[i].reshape(1,128), sentence_vectors[j].reshape(1,128))[0,0]
    nx_graph = nx.from_numpy_array(sim_mat)
    scores = nx.pagerank(nx_graph)
    ranked_sentences = sorted(((scores[i],s) for i,s in enumerate(og_sentences)), reverse=True)
    summary = ''
    for i in range(7):
        summary += ' ' + ranked_sentences[i][1]
    return summary

In [13]:
nx_graph = nx.from_numpy_array(sim_mat)
scores = nx.pagerank(nx_graph)
ranked_sentences = sorted(((scores[i],s) for i,s in enumerate(og_sentences)), reverse=True)
for i in range(7):
    print(i, '--> ',ranked_sentences[i][1])

0 -->  And eventually after a couple of weeks of travel that probably felt like years Onesimus and Tychicus arrive in Colossae and Onesimus comes face to face with Philemon and hes probably terrified but he has this letter and it would be sealed with Pauls personal seal and its signed in Pauls hand so that Philemon would know its the real deal.
1 -->  I thank my God always when I remember you in my prayers because I hear of your love and of the faith that you have toward the Lord Jesus and for all the saints and I pray that the sharing of your faith may become effective for the full knowledge of every good thing that is in us for the sake of Christ.
2 -->  But they would likely have been slaves so to become a bishop seems unlikely... it makes sense that it might be our Onesimus the one who spent significant time with Paul during his imprisonment in Rome who would have gone back to Paul in Rome if Philemon received his letter and did the more than Paul even asked.
3 -->  So its one thin